# **SDG Prediction Validation**

## **Dependencies**

In [21]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## **Load Data**

### Load GIZ Project Data As Ground Truth

GIZ Running Projects can be used as ground truth for SDG.

In [2]:
running_giz_df = pd.read_excel("../../src/ground_truths/giz_running_projects.xlsx")

running_giz_df.head(1)

,*Projektnummer,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt
0,2008.9076.4,Unterstützung bei der Einführung eines Kühlsch...,Refrigerator recycling system,41010.0,Umweltpolitik und -verwaltung,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13;15;,Unterstützung beim Aufbau eines beispielhaften...,Supporting the introduction of an exemplary ta...,"Umweltpolitik, Schutz und nachhaltige Nutzung ..."


### Load Merged DF

In [3]:
df = pd.read_csv("../../src/merged_orgas.csv")
df.head(1)

C:\Users\UserNA6153\AppData\Local\Temp\ipykernel_15396\34694008.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../src/merged_orgas.csv")


,iati_id,iati_orga_id,orga_abbreviation,orga_full_name,project_number,title_en,title_other,title_main,organization,country_code,...,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs,sgd_pred_code,sgd_pred_str
0,DE-1-199535469,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,1,Studies and Experts Fund,Studien- und Fachkräftefonds,Studies and Experts Fund,Bundesministerium für wirtschaftliche Zusammen...,['NA'],...,2026-12-31T00:00:00Z,2024-01-31T00:00:00Z,2024-01-31T00:00:00Z,43010;,Multisector aid;,430;,Other Multisector;,['https://www.giz.de/projektdaten/projects.act...,15.0,"Goal 15. Protect, restore and promote sustaina..."


### Extract Project Numbers out of IATI ID

In [4]:
bmz_df = df[df.iati_orga_id == "DE-1"]

bmz_df["pn"] = "NaN"

for index, row in bmz_df.iterrows():
    try:
        pn_raw = row['iati_id'].split("-")[2]
        pn = f"{pn_raw[:4]}.{pn_raw[4:8]}.{pn_raw[8:]}"
        bmz_df.loc[index, "pn"] = pn
    except:
        pass

bmz_df.head(2)

C:\Users\UserNA6153\AppData\Local\Temp\ipykernel_15396\514410831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bmz_df["pn"] = "NaN"


,iati_id,iati_orga_id,orga_abbreviation,orga_full_name,project_number,title_en,title_other,title_main,organization,country_code,...,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs,sgd_pred_code,sgd_pred_str,pn
0,DE-1-199535469,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,1,Studies and Experts Fund,Studien- und Fachkräftefonds,Studies and Experts Fund,Bundesministerium für wirtschaftliche Zusammen...,['NA'],...,2024-01-31T00:00:00Z,2024-01-31T00:00:00Z,43010;,Multisector aid;,430;,Other Multisector;,['https://www.giz.de/projektdaten/projects.act...,15.0,"Goal 15. Protect, restore and promote sustaina...",1995.3546.9
1,DE-1-199665852,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,1,Windhoek Water Reclamation,Abwasserrückgewinnung Windhoek,Windhoek Water Reclamation,Bundesministerium für wirtschaftliche Zusammen...,['NA'],...,2024-01-31T00:00:00Z,2024-01-31T00:00:00Z,14020;,Water supply and sanitation - large systems;,140;,Water Supply & Sanitation;,['https://www.kfw-entwicklungsbank.de/Internat...,12.0,Goal 12. Ensure sustainable consumption and pr...,1996.6585.2


### Extract running GIZ projects just with one sdg to achieve better compareability 

In [5]:
running_giz_df = running_giz_df[running_giz_df['SDG Hauptziele'].str.len() == 3]

print(f"{len(running_giz_df)} with 1 SDG")
running_giz_df.head(1)

521 with 1 SDG


,*Projektnummer,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt
1,2010.9226.1,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU ...,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU b...,99810.0,Nicht spezifizierte Sektoren (fortgeführt als ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17;,EU Twinning,EU Twinning,Gestaltungsspielraum


### Match all running giz projects with IATI BMZ data 

In [6]:
# rename project number column in running_giz_df
running_giz_df.rename(columns={'*Projektnummer': 'pn'}, inplace=True)
running_giz_df.head(1)

,pn,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt
1,2010.9226.1,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU ...,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU b...,99810.0,Nicht spezifizierte Sektoren (fortgeführt als ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17;,EU Twinning,EU Twinning,Gestaltungsspielraum


In [8]:
# match with project number
merged_df = pd.merge(running_giz_df, bmz_df[['pn', 'sgd_pred_code']], on='pn', how='left')
merged_df.dropna(subset=['sgd_pred_code'], inplace=True)
print(f"{len(merged_df)}")
merged_df.head(5)

420


,pn,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt,sgd_pred_code
3,2014.0968.9,Globalvorhaben Ernährungssicherung und Resilie...,"Global programme Food and Nutrition Security, ...",43072.0,Programme zur Ernährungssicherung auf Haushalt...,0.7,43071.0,Politiken/Verwaltungsführung zur Ernährungssic...,0.3,NaN,NaN,NaN,NaN,NaN,NaN,02;,Die Ernährungssituation und die Resilienz gege...,The food situation and the resilience to hunge...,"Sicherung der Ernährung, Landwirtschaft",8.0
4,2014.2267.4,Modernisierung kommunaler Dienstleistungen,Modernization of Local Public Services,43030.0,Stadtentwicklung und -verwaltung,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11;,In ländlichen Gemeinden werden kommunale Diens...,Efficient and sustainable provision of local p...,"Demokratie, Zivilgesellschaft, und öffentliche...",3.0
5,2014.2267.4,Modernisierung kommunaler Dienstleistungen,Modernization of Local Public Services,43030.0,Stadtentwicklung und -verwaltung,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11;,In ländlichen Gemeinden werden kommunale Diens...,Efficient and sustainable provision of local p...,"Demokratie, Zivilgesellschaft, und öffentliche...",3.0
6,2014.2267.4,Modernisierung kommunaler Dienstleistungen,Modernization of Local Public Services,43030.0,Stadtentwicklung und -verwaltung,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11;,In ländlichen Gemeinden werden kommunale Diens...,Efficient and sustainable provision of local p...,"Demokratie, Zivilgesellschaft, und öffentliche...",3.0
7,2014.2267.4,Modernisierung kommunaler Dienstleistungen,Modernization of Local Public Services,43030.0,Stadtentwicklung und -verwaltung,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11;,In ländlichen Gemeinden werden kommunale Diens...,Efficient and sustainable provision of local p...,"Demokratie, Zivilgesellschaft, und öffentliche...",3.0


In [11]:
# just use most important columns
analy_df = merged_df[["pn", "SDG Hauptziele", "sgd_pred_code"]]
analy_df.head(400)

,pn,SDG Hauptziele,sgd_pred_code
3,2014.0968.9,02;,8.0
4,2014.2267.4,11;,3.0
5,2014.2267.4,11;,3.0
6,2014.2267.4,11;,3.0
7,2014.2267.4,11;,3.0
...,...,...,...
658,2023.2125.5,16;,3.0
659,2023.2127.1,04;,14.0
667,2023.2169.3,08;,14.0
668,2023.2169.3,08;,14.0


In [14]:
# change dtype that both cols can be compared
analy_df["SDG Hauptziele"] = analy_df["SDG Hauptziele"].str[:2].astype(float)

analy_df.head()

C:\Users\UserNA6153\AppData\Local\Temp\ipykernel_15396\193875488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analy_df["SDG Hauptziele"] = analy_df["SDG Hauptziele"].str[:2].astype(float)


,pn,SDG Hauptziele,sgd_pred_code
3,2014.0968.9,2.0,8.0
4,2014.2267.4,11.0,3.0
5,2014.2267.4,11.0,3.0
6,2014.2267.4,11.0,3.0
7,2014.2267.4,11.0,3.0


## **Analysis**

### Compare ground truth with predicted

In [24]:
cm = confusion_matrix(analy_df['SDG Hauptziele'], analy_df['sgd_pred_code'])
print(cm)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  5  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 13  0  0  0  0  0  1  0]
 [ 0  0  0  0  2  0  0  0  0 29  3  0  6 36 15  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  1  3  6  2  0  0  0  1  1  3  0  0 82 29  0]
 [ 0  0  0  0  1  0  0  0  0  1  0  2  4  0  3  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  8  2  0]
 [ 4  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  7  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  5  0  2  0  0  1  0  0  0  0]
 [ 6  2 41  2  5  0  1  0  3  0  9  2  0 15 20  0]]
